In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

In [2]:
# convert images to PyTorch tensors
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
data_dir = '/exchange/dspro01/group3/data/train' 
dataset = datasets.ImageFolder(root=data_dir, transform=transform) # assigns labels automatically according to folder's name


In [4]:
num_classes = 12            # number of categories
num_epochs =  10           # set the number of epochs for training
learning_rate = 0.001 

In [5]:
# For reproducibility
seed = 42
torch.manual_seed(seed)

In [6]:
total_size = len(dataset)
train_size = int(0.8 * total_size)   
val_size = int(0.2 * total_size)    

In [7]:
# Split
generator = torch.Generator().manual_seed(seed) # reproducible
train_dataset, val_dataset = random_split(
    dataset, [train_size, val_size], generator=generator
) 

In [8]:
len(train_dataset) == (2400 * 12 * 0.8)

True

In [9]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [13]:
from torchvision import models
from torchvision.models import EfficientNet_B0_Weights
import torch.nn as nn
import torch.optim as optim
import torch

In [14]:
# Load EfficientNet model with weights
weights = EfficientNet_B0_Weights.IMAGENET1K_V1  # Load pre-trained weights
model = models.efficientnet_b0(weights=weights)

In [15]:
# Modify the classifier to match the number of classes (12 in this case)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 12)

In [16]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

In [17]:
# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Adjust learning rate if necessary


In [18]:
# Define a learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [19]:
# Move the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [20]:
# Dataloader dictionary (ensure train_loader and val_loader are defined)
dataloaders = {'train': train_loader, 'val': val_loader}


In [21]:
# Dataset sizes (ensure train_dataset and val_dataset are defined)
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}


In [22]:
# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluation mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass and optimization (only in training phase)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate epoch loss and accuracy
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        if phase == 'train':
            scheduler.step()

print('Training complete')

Epoch 1/10
----------
train Loss: 0.8497 Acc: 0.7080
val Loss: 0.6185 Acc: 0.7903
Epoch 2/10
----------
train Loss: 0.5362 Acc: 0.8163
val Loss: 0.5074 Acc: 0.8306
Epoch 3/10
----------
train Loss: 0.4248 Acc: 0.8529
val Loss: 0.4515 Acc: 0.8479
Epoch 4/10
----------
train Loss: 0.3503 Acc: 0.8799
val Loss: 0.4736 Acc: 0.8415
Epoch 5/10
----------
train Loss: 0.3012 Acc: 0.8969
val Loss: 0.4734 Acc: 0.8521
Epoch 6/10
----------
train Loss: 0.1404 Acc: 0.9521
val Loss: 0.3833 Acc: 0.8941
Epoch 7/10
----------
train Loss: 0.0828 Acc: 0.9733
val Loss: 0.4123 Acc: 0.8929
Epoch 8/10
----------
train Loss: 0.0573 Acc: 0.9826
val Loss: 0.4361 Acc: 0.8972
Epoch 9/10
----------
train Loss: 0.0435 Acc: 0.9865
val Loss: 0.4823 Acc: 0.8955
Epoch 10/10
----------
train Loss: 0.0343 Acc: 0.9895
val Loss: 0.4988 Acc: 0.8953
Training complete
